In [1]:
import pandas as pd
import tabula
import os

In [35]:
import pandas as pd
import pdfplumber
import os
import re

def extract_tables_pdfplumber(pdf_path, filename):
    """
    Extrait les tables d'un PDF en utilisant pdfplumber, ajoute le nom du fichier, et les retourne sous forme de liste de dataframes.
    Args:
        pdf_path (str): Le chemin vers le fichier PDF.
        filename (str): Le nom du fichier PDF (sans le chemin).
    Returns:
        list: Une liste de DataFrames extraits du PDF. Si aucun tableau n'est trouvé, retourne une liste vide.
    """
    dfs = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                for table in page.extract_tables():
                    df = pd.DataFrame(table[1:], columns=table[0])
                    df['filename'] = filename
                    dfs.append(df)
    except Exception as e:
        print(f"Erreur lors du traitement du fichier {pdf_path}: {e}")
        return []
    return dfs

def process_pdfs_in_folder_pdfplumber(folder_path):
    """
    Parcourt tous les fichiers PDF dans un dossier, extrait les tableaux,
    ajoute le nom du fichier, sélectionne certaines colonnes, extrait départ/arrivée, supprime le détail voyage,
    convertit les colonnes numériques et date en bon format et les fusionne en un seul dataframe.

    Args:
        folder_path (str): Le chemin vers le dossier contenant les fichiers PDF.

    Returns:
        pandas.DataFrame: Un dataframe contenant tous les tableaux extraits des PDF fusionnés, ou None si aucun tableau n'est trouvé dans le dossier
    """
    all_dfs = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf") and filename.startswith("SupplierPreInvoiceSPI"):
            file_path = os.path.join(folder_path, filename)
            dfs = extract_tables_pdfplumber(file_path, filename)
            if dfs:
                for df in dfs:
                   try:
                        # Selectionner colonnes
                        df = df[['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:', 'filename']]

                         # Extraction ville et codes postaux dans "Depart" et "Arrivee"
                        df['Depart'] = df['Détail voyage:'].str.extract(r'Départ: (FR-\d{5} [A-ZÀ-Ÿ\s-]*)')
                        df['Depart'] = df['Depart'].str.replace("Départ: ","").str.strip()
                        df['Arrivee'] = df['Détail voyage:'].str.extract(r'Arrivée: (FR-\d{5} [A-ZÀ-Ÿ\s-]*)')
                        df['Arrivee'] = df['Arrivee'].str.replace("Arrivée: ", "").str.strip()
                          # Nettoyage Voyage N°
                        df['Voyage N°:'] = df['Voyage N°:'].str.split('\n').str[0].str.strip()

                           # Suppression de la colonne Detail Voyage
                        df = df.drop('Détail voyage:', axis=1)
                   
                        # Conversion des numeriques avec 'coerce'
                        df['Exp.Date:'] = pd.to_datetime(df['Exp.Date:'], format='%d/%m/%Y', errors='coerce')


                        all_dfs.append(df)

                   except KeyError as e:
                        print(f"Attention : Colonne(s) manquante(s) dans un PDF : {e}")

    if not all_dfs:
        print("Aucun tableau n'a été trouvé dans les PDF.")
        return None

    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df

if __name__ == '__main__':
    folder_path = r'C:\Projet 3\downloads'  # Remplacez ceci par votre chemin
    
    combined_dataframe = process_pdfs_in_folder_pdfplumber(folder_path)

    if combined_dataframe is not None:
       print(combined_dataframe.dtypes)
       print(combined_dataframe) #verification que c'est bon
       combined_dataframe.to_csv("extracted_data_pdfplumber.csv", index=False)
       print("Le dataframe a été enregistré dans le fichier extracted_data_pdfplumber.csv")
    else:
       print("Aucun dataframe n'a été créé")

Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF : "['Exp.Date:', 'Voyage N°:', 'Détail voyage:', 'Poids:', 'Montant HT:'] not in index"
Attention : Colonne(s) manquante(s) dans un PDF 

In [43]:
df = pd.read_csv(r"C:\Projet 3\extracted_data_pdfplumber.csv")
df.head()


,Exp.Date:,Voyage N°:,Poids:,Montant HT:,filename,Depart,Arrivee
0,2023-12-15,PARTRP000100992,"6000,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-77185 LOGNES\nA,FR-60300 SENLIS
1,2023-12-15,PARTRP000101238,"769,91","260,00",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-95130 LE PLESSIS
2,2023-12-18,PARTRP000101276,"184,62","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-77400 ST THIBAULT
3,2023-12-18,PARTRP000101277,"202,33","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-91310 MONTLHERY
4,2023-12-19,PARTRP000101305,"3160,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-94200 IVRY SUR SEINE\nA,FR-60300 SENLIS


In [17]:
df.shape

(807, 7)

In [38]:
#df['Poids:'] = pd.to_numeric(df['Poids:'], errors='coerce')
#df['Montant HT:'] = pd.to_numeric(df['Montant HT:'], errors='coerce')

In [39]:
df['Exp.Date:'] = pd.to_datetime(df['Exp.Date:'], format='%d/%m/%Y', errors='coerce')

In [44]:
df.head()

,Exp.Date:,Voyage N°:,Poids:,Montant HT:,filename,Depart,Arrivee
0,2023-12-15,PARTRP000100992,"6000,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-77185 LOGNES\nA,FR-60300 SENLIS
1,2023-12-15,PARTRP000101238,"769,91","260,00",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-95130 LE PLESSIS
2,2023-12-18,PARTRP000101276,"184,62","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-77400 ST THIBAULT
3,2023-12-18,PARTRP000101277,"202,33","267,50",SupplierPreInvoiceSPI00450803.pdf,FR-95500 BONNEUIL EN\nA,FR-91310 MONTLHERY
4,2023-12-19,PARTRP000101305,"3160,00","350,00",SupplierPreInvoiceSPI00450803.pdf,FR-94200 IVRY SUR SEINE\nA,FR-60300 SENLIS


In [22]:
df.describe()

,Exp.Date:,Voyage N°:,Poids:,Montant HT:,filename,Depart,Arrivee
count,807,807,807,807,807,807,797
unique,235,781,769,106,50,47,253
top,31/01/2024,LILTRP000132604,"1000,00","270,00",SupplierPreInvoiceSPI00512502.pdf,FR-95500 BONNEUIL EN\nA,FR-60300 SENLIS
freq,9,3,3,50,40,444,58
